In [8]:
import os

In [9]:
os.chdir("../")                           

In [14]:
os.chdir('c:\\Users\\yashr\\OneDrive\\Desktop\\Text_Summarization_project\\Text-summarizer-app')

In [15]:
%pwd

'c:\\Users\\yashr\\OneDrive\\Desktop\\Text_Summarization_project\\Text-summarizer-app'

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path



In [23]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml,create_directories



In [28]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config
    

In [25]:
import os 
import urllib.request as request
import zipfile

from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size



In [26]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig) -> None:
        self.config = config

    

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following info : \n{headers}")
        
        else:
            logger.info(f"File already exists of size :{get_size(Path(self.config.local_data_file))}")
    

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
              
        


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e
